In [60]:
####################################################################
# importing the module
import pandas as pd
import numpy as np
from os.path import join as opj
import shapely as sp
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame

DATA_DIR = "../data"    
MSW_gross = "feedstocks_raw/CA_grossMSW20.csv"
HMS_gross = "feedstocks_raw/CA_grossHMS20.csv"
LMS_gross = "feedstocks_raw/CA_grossLMS20.csv"
manure_gross = "feedstocks_raw/CA_grossmanure.csv"
orchres_gross = "feedstocks_raw/CA_grossorchres20.csv"
rowres_gross = "feedstocks_raw/CA_grossrowres20.csv"

# reading in the data files
MSW = pd.read_csv(opj(DATA_DIR, MSW_gross))    
#sites = pd.read_excel("../data/swis/Sites.xlsx")
HMS = pd.read_csv(opj(DATA_DIR, HMS_gross)) 
LMS = pd.read_csv(opj(DATA_DIR, LMS_gross))
manure = pd.read_csv(opj(DATA_DIR, manure_gross))
orchres = pd.read_csv(opj(DATA_DIR, orchres_gross))
rowres = pd.read_csv(opj(DATA_DIR, rowres_gross))

In [61]:
###from Anaya's code to turn into a function---CAN DELETE###
#def MergeInventoryAndCounty(df, county_shapefile, counties_popcen):
    #"""
        #Cleans biomass inventory data and merges with county shapefiles
        #gross_inventory      - gross estimate of biomass inventory
        #technical_inventory  - technical estimate of biomass inventory ##maybe not here
        #county_shapefile     - shapefile of county polygons
        #counties_popcen      - csv population-weighted county centroids

        #Returns: cleaned, spatial biomass data (assigned to pop-weighted 
        #county centroids and county polygons)
    #"""

    ##################################################################

In [63]:
###CLEANING MSW FILE###

MSW
#remove columns for "high recycle" - we will assume baseline
new_MSW = MSW.drop(columns = ['Unnamed: 2', 'Unnamed: 4', 'Unnamed: 6', 'Unnamed: 8', 'Unnamed: 10', 'Unnamed: 12', 'Unnamed: 14'])
new_MSW
#remove unncessary rows
MSW_FINAL = new_MSW.drop([new_MSW.index[0], new_MSW.index[60]])
MSW_FINAL = MSW_FINAL.rename(columns={" COUNTY ":"COUNTY"})

#remove extra spaces
for (i, row) in MSW_FINAL.iterrows():
    row.COUNTY = row.COUNTY.strip()
    
# all of these values are strings! want as int for using in the model
# had to go back into excel to change the format from number to general to get rid of commas, 
# and replace missing data values from " - " to "0"
# df['DataFrame Column'] = df['DataFrame Column'].astype(int)
MSW_FINAL['GREEN'] = MSW_FINAL['GREEN'].astype(int)
MSW_FINAL['FOOD'] = MSW_FINAL['FOOD'].astype(int)
MSW_FINAL['PAPER'] = MSW_FINAL['PAPER'].astype(int)
MSW_FINAL['CARDBOARD'] = MSW_FINAL['CARDBOARD'].astype(int)
MSW_FINAL['LUMBER'] = MSW_FINAL['LUMBER'].astype(int)

#also while i'm at it, adding a column of all MSW in case it's useful later 
MSW_FINAL['MSW_SUM'] = MSW_FINAL.GREEN + MSW_FINAL.LUMBER + MSW_FINAL.FOOD + MSW_FINAL.PAPER + MSW_FINAL.CARDBOARD

MSW_FINAL

,COUNTY,LUMBER,PAPER,CARDBOARD,GREEN,FOOD,FOG,OTHER,MSW_SUM
1,Alameda,147821,127884,33169,94768,43944,156,93735,447586
2,Alpine,114,161,42,119,55,0,119,491
3,Amador,3436,2971,771,2202,1021,4,2177,10401
4,Butte,20819,20190,5237,14962,6938,22,14868,68146
5,Calaveras,4302,3574,927,2649,1228,5,2591,12680
6,Colusa,2134,2391,620,1772,822,2,1720,7739
7,Contra Costa,102511,79163,20532,58663,27202,108,58003,288071
8,Del Norte,2561,1846,479,1368,634,3,1387,6888
9,El Dorado,16769,15453,4008,11452,5310,18,11492,52992
10,Fresno,92708,84538,21926,62647,29049,98,60903,290868


In [64]:
###FINISH MSW CLEANING###
dataframe = MSW_FINAL
print(dataframe.columns)

#inner merge excludes state totals and "Other" category for manure that is not included in individual counties
#switch to outer if want to include
dataframe

selected_columns = dataframe[["COUNTY", "LUMBER", "PAPER", "CARDBOARD", "GREEN", "FOOD", "FOG", "OTHER"]]

df = selected_columns.copy()
df

#convert to wet tons#
fw_mc = 0.7
gw_mc = 0.5

#fix up food column
for (i, row) in df.iterrows():
    if type(row.FOOD) != int:
        row.FOOD = int(row.FOOD.replace("," , "").replace("-", "0"))

#fix up green column
for (i, row) in df.iterrows():
    if type(row.GREEN) != int:
        row.GREEN = int(row.GREEN.replace("," , "").replace("-", "0"))

#complete bdt to wet ton conversions - add new columns
df['food_wt'] = df['FOOD'] / (1- fw_mc)

df['gw_wt'] = df['GREEN'] / (1-gw_mc)

df 

#DF IS THE DATAFRAME FOR MSW - FOOD AND GREEN WASTE



Index(['COUNTY', 'LUMBER', 'PAPER', 'CARDBOARD', 'GREEN', 'FOOD', 'FOG',
       'OTHER', 'MSW_SUM'],
      dtype='object')


,COUNTY,LUMBER,PAPER,CARDBOARD,GREEN,FOOD,FOG,OTHER,food_wt,gw_wt
1,Alameda,147821,127884,33169,94768,43944,156,93735,1.464800e+05,189536.0
2,Alpine,114,161,42,119,55,0,119,1.833333e+02,238.0
3,Amador,3436,2971,771,2202,1021,4,2177,3.403333e+03,4404.0
4,Butte,20819,20190,5237,14962,6938,22,14868,2.312667e+04,29924.0
5,Calaveras,4302,3574,927,2649,1228,5,2591,4.093333e+03,5298.0
6,Colusa,2134,2391,620,1772,822,2,1720,2.740000e+03,3544.0
7,Contra Costa,102511,79163,20532,58663,27202,108,58003,9.067333e+04,117326.0
8,Del Norte,2561,1846,479,1368,634,3,1387,2.113333e+03,2736.0
9,El Dorado,16769,15453,4008,11452,5310,18,11492,1.770000e+04,22904.0
10,Fresno,92708,84538,21926,62647,29049,98,60903,9.683000e+04,125294.0


In [65]:
###START CLEANING MANURE FILE###
manure

print(manure.columns)

#Code below is if we want to merge manure file with MSW data above - didn't use it

#for file in [manure]:
    #dataframe = pd.merge(dataframe, file, on = "COUNTY", how = "inner") 
    
manure = manure.rename(columns={"TOTAL MANURE":"TOTAL_MANURE"})

selected_columns = manure[["COUNTY", "TOTAL_MANURE"]]

manure = selected_columns.copy()

manure

Index(['COUNTY', 'TOTAL MANURE', 'COWSMILK', 'COWSBEEF', 'COWSOTHER',
       ' LAYERS ', ' PULLETSlAYERREPL ', ' BROILERS ', ' TURKEYS ',
       ' HOGSPIGS ', ' GOATS ', ' SHEEPLAMBS ', ' EQUINE ',
       ' MULESBURROSDONKEY '],
      dtype='object')


,COUNTY,TOTAL_MANURE
0,Alameda,"30,612"
1,Alpine,596
2,Amador,"28,288"
3,Butte,"31,633"
4,Calaveras,"30,616"
5,Colusa,"29,581"
6,Contra Costa,"43,371"
7,Del Norte,"18,507"
8,El Dorado,"21,222"
9,Fresno,"829,062"


In [66]:
###KEEP CLEANING MANURE DATA###

#conversion to wet tons
manure_mc = 0.85

#fix up manure column
for (i, row) in manure.iterrows():
    if type(row.TOTAL_MANURE) != int:
        row.TOTAL_MANURE = int(row.TOTAL_MANURE.replace("," , "").replace("-", "0"))
    
manure['manure_wt'] = manure['TOTAL_MANURE'] / (1-manure_mc)

manure

,COUNTY,TOTAL_MANURE,manure_wt
0,Alameda,30612,204080
1,Alpine,596,3973.33
2,Amador,28288,188587
3,Butte,31633,210887
4,Calaveras,30616,204107
5,Colusa,29581,197207
6,Contra Costa,43371,289140
7,Del Norte,18507,123380
8,El Dorado,21222,141480
9,Fresno,829062,5.52708e+06


In [67]:
###START CLEANING HIGH MOISTURE SOLIDS FROM FOOD PROCESSORS FILE###

HMS

print(HMS.columns)

HMS = HMS.rename(columns={" HMS TOTAL ":"HMS_TOTAL", " COUNTY ": "COUNTY"})

selected_columns = HMS[["COUNTY", "HMS_TOTAL"]]

HMS = selected_columns.copy()

HMS = HMS.drop([HMS.index[60]])



Index([' COUNTY ', ' HMS TOTAL ', 'CANNERY', 'DEHYD.', 'FRESH/FROZ.', 'WINERY',
       'BREWERY', 'DISTILLERY', 'RED MEAT', 'POULTRY MEAT', 'Unnamed: 10'],
      dtype='object')


In [68]:
###FINISH CLEAN UP OF HMS NUMBERS###

#CONVERSION TO WET TONS
hms_mc = 0.78

#got this number from an Breunig paper from 70-78% MC content range for food waste

for (i, row) in HMS.iterrows():
    if type(row.HMS_TOTAL) != int:
        row.HMS_TOTAL = int(row.HMS_TOTAL.replace("," , "").replace("-", "0"))

for (i, row) in HMS.iterrows():
    row.COUNTY = row.COUNTY.strip()
        
HMS['HMS_wt'] = HMS['HMS_TOTAL'] / (1-hms_mc)

HMS

print(HMS.columns)


Index(['COUNTY', 'HMS_TOTAL', 'HMS_wt'], dtype='object')


In [69]:
###START CLEANING LOW MOISTURE SOLIDS FROM FOOD AND FIBER PROCESSORS FILE###

LMS

print(LMS.columns)

LMS = LMS.rename(columns={" LMS TOTAL ":"LMS_TOTAL", " COUNTY ": "COUNTY"})

selected_columns = LMS[["COUNTY", "LMS_TOTAL"]]

LMS = selected_columns.copy()

LMS = LMS.drop([LMS.index[60]])

LMS

Index([' COUNTY ', ' LMS TOTAL ', 'ALMOND HULLS', 'ALMOND SHELLS',
       'WALNUT SHELLS', 'OTHER NUT SHELLS', 'RICE HULLS', 'COTTON GIN TRASH',
       'CANNERY', 'DEHYD.', 'BAKERY', 'TORTILLA', 'GRAIN CHAFF'],
      dtype='object')


,COUNTY,LMS_TOTAL
0,Alameda,"5,637"
1,Alpine,-
2,Amador,51
3,Butte,"198,107"
4,Calaveras,316
5,Colusa,"414,624"
6,Contra Costa,"1,831"
7,Del Norte,-
8,El Dorado,15
9,Fresno,"900,162"


In [70]:
###FINISH CLEAN UP OF LMS NUMBERS###

#CONVERSION TO WET TONS

lms_mc = 0.13

#used an average of nut, rice, cotton processors MC from Breunig "additional methods"

for (i, row) in LMS.iterrows():
    if type(row.LMS_TOTAL) != int:
        row.LMS_TOTAL = int(row.LMS_TOTAL.replace("," , "").replace("-", "0"))
        
for (i, row) in LMS.iterrows():
    row.COUNTY = row.COUNTY.strip()
        
LMS['LMS_wt'] = LMS['LMS_TOTAL'] / (1-lms_mc)

LMS


,COUNTY,LMS_TOTAL,LMS_wt
0,Alameda,5637,6479.31
1,Alpine,0,0
2,Amador,51,58.6207
3,Butte,198107,227709
4,Calaveras,316,363.218
5,Colusa,414624,476579
6,Contra Costa,1831,2104.6
7,Del Norte,0,0
8,El Dorado,15,17.2414
9,Fresno,900162,1.03467e+06


In [71]:
###START CLEANING ORCHARD RESIDUE SOLIDS FILE###

orchres

print(orchres.columns)

orchres = orchres.rename(columns={" TOTAL ORCH. VINE CROP ":"ORCHRES_TOTAL", " COUNTY ": "COUNTY"})

selected_columns = orchres[["COUNTY", "ORCHRES_TOTAL"]]

orchres = selected_columns.copy()

orchres = orchres.drop([orchres.index[60]])

orchres


Index([' COUNTY ', ' TOTAL ORCH. VINE CROP ', ' ALMOND ', ' GRAPE ', ' APPLE ',
       ' APRICOT ', ' AVOCADO ', ' EXOTIC ', ' CHERRY ', ' NUTS OTHER ',
       ' LEMON ', ' CITRUS OTHER ', ' DATE & FIG ', ' FRUITS & NUTS UNSP. ',
       ' KIWIFRUIT ', ' NECTARINE ', ' OLIVE ', ' ORANGE ', ' PEACH ',
       ' PEAR ', ' PISTACHIO ', ' PLUM ', ' WALNUT '],
      dtype='object')


,COUNTY,ORCHRES_TOTAL
0,Alameda,"2,659"
1,Alpine,-
2,Amador,"4,109"
3,Butte,"89,497"
4,Calaveras,"1,480"
5,Colusa,"105,105"
6,Contra Costa,"3,414"
7,Del Norte,-
8,El Dorado,"3,659"
9,Fresno,"655,400"


In [72]:
###FINISH CLEAN UP OF ORCHARD RESIDUE NUMBERS###

#CONVERSION TO WET TONS

orch_mc = 0.42

#got this number by averaging MC for orchard/vineyard residues in Breunig "additional methods"

for (i, row) in orchres.iterrows():
    if type(row.ORCHRES_TOTAL) != int:
        row.ORCHRES_TOTAL = int(row.ORCHRES_TOTAL.replace("," , "").replace("-", "0"))

for (i, row) in orchres.iterrows():
    row.COUNTY = row.COUNTY.strip()
        
orchres['ORCHRES_wt'] = orchres['ORCHRES_TOTAL'] / (1-lms_mc)

orchres

print(orchres.columns)



Index(['COUNTY', 'ORCHRES_TOTAL', 'ORCHRES_wt'], dtype='object')


In [73]:
####ROW CROP RESIDUE####

###going to leave out for now because there is not much data available###
###we are also leaving out field crops in this analysis - can bring in if interesting###

rowres

print(rowres.columns)

rowres = rowres.rename(columns={" TOTAL      ROW CROP ":"ROWRES_TOTAL", " COUNTY ": "COUNTY"})

selected_columns = rowres[["COUNTY", "ROWRES_TOTAL"]]

rowres = selected_columns.copy()

rowres = rowres.drop([rowres.index[60]])

rowres

Index([' COUNTY ', ' TOTAL      ROW CROP ', ' ARTICHOKE ', ' ASPARAGUS ',
       ' BEET ', ' BERRY ', ' BROCCOLI ', ' CABBAGE  ', ' CARROT ',
       ' CAULIFLOWER ', ' CELERY ', ' CUCUMBER ', ' GARLIC ', ' LETTUCE ',
       ' MELON ', ' ONION ', ' PEPPER ', ' POTATO ', ' SQUASH ',
       ' SPINACH & GREENS ', ' SPICES & HERBS ', ' TOMATO& EGGPLANT ',
       ' VEG UNSP. ', ' OTHER CRUCIFEROUS ', ' VEG OTHER ', ' BEAN & PEA ',
       ' CORN SWEET '],
      dtype='object')


,COUNTY,ROWRES_TOTAL
0,Alameda,28
1,Alpine,-
2,Amador,-
3,Butte,2
4,Calaveras,-
...,...,...
119,NaN,NaN
120,NaN,NaN
121,NaN,NaN
122,NaN,NaN


In [74]:
 # # now load SHAPEFILE for all county_shape COUNTIES to merge this
    # print("p Read in county_shape COUNTIES shapefile and reproject")
california_county_shape = "CA_Counties/CA_Counties_TIGER2016.shp"
county_shape = gpd.read_file(opj(DATA_DIR, california_county_shape))
county_shape.rename(columns = {'NAME': 'COUNTY'}, inplace=True)
county_shape= county_shape.to_crs(epsg=4326)
county_shape.crs

county_shape['county_centroid'] = county_shape['geometry'].centroid 

selected_columns = county_shape[["COUNTY", "GEOID", "county_centroid"]]
county_shape = selected_columns.copy()
county_shape


/Users/anayahall/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  if __name__ == '__main__':


,COUNTY,GEOID,county_centroid
0,Sierra,06091,POINT (-120.51601 39.58041)
1,Sacramento,06067,POINT (-121.34426 38.44932)
2,Santa Barbara,06083,POINT (-120.03080 34.53835)
3,Calaveras,06009,POINT (-120.55415 38.20461)
4,Ventura,06111,POINT (-119.12604 34.35753)
5,Los Angeles,06037,POINT (-118.26103 34.19801)
6,Sonoma,06097,POINT (-122.92256 38.52530)
7,Kings,06031,POINT (-119.81552 36.07537)
8,San Diego,06073,POINT (-116.77022 33.02821)
9,Placer,06061,POINT (-120.71766 39.06343)


In [86]:
# ALSO LOAD IN CSV of population-weighted county centroids - use this not geographic centroid for MSW!!
countieswithpopcentroid = "CenPop2010.csv"
counties_popcen = pd.read_csv(opj(DATA_DIR, countieswithpopcentroid)) # NEW - population weighted means!
counties_popcen.rename(columns = {'LATITUDE': 'lat', 'LONGITUDE': 'lon', 'COUNAME': 'COUNTY'}, inplace=True)

counties_popcen['county_centroid'] = [sp.geometry.Point(xy) for xy in 
            zip(counties_popcen.lon, counties_popcen.lat)]

counties_popcen
#geometry = [Point(xy) for xy in zip(counties_popcen.lon, counties_popcen.lat)]

,STATEFP,COUNTYFP,COUNTY,STNAME,POPULATION,lat,lon,county_centroid
0,6,1,Alameda,California,1510271,37.705201,-122.100393,POINT (-122.100393 37.705201)
1,6,3,Alpine,California,1175,38.728298,-119.847603,POINT (-119.847603 38.728298)
2,6,5,Amador,California,38091,38.394208,-120.775729,POINT (-120.775729 38.394208)
3,6,7,Butte,California,220000,39.669037,-121.707635,POINT (-121.707635 39.669037)
4,6,9,Calaveras,California,45578,38.163627,-120.633718,POINT (-120.633718 38.16362700000001)
5,6,11,Colusa,California,21419,39.156148,-122.085557,POINT (-122.085557 39.15614799999999)
6,6,13,Contra Costa,California,1049025,37.935588,-122.030131,POINT (-122.030131 37.935588)
7,6,15,Del Norte,California,28610,41.791232,-124.163088,POINT (-124.163088 41.791232)
8,6,17,El Dorado,California,181058,38.746543,-120.744330,POINT (-120.74433 38.746543)
9,6,19,Fresno,California,930450,36.738918,-119.767884,POINT (-119.767884 36.738918)


In [87]:
 #COUNTY POLYGONS with MSW DATA(mostly for plotting - don't use this one for actual optimization)
df_shp = pd.merge(county_shape, df, on = 'COUNTY')

#COUNTY POLYGONS with MANURE DATA (using geographic center, not population)
manure_shp = pd.merge(county_shape, manure, on = 'COUNTY')

#COUNTY POLYGONS with HIGH MOISTURE DATA (using geographic center, not population)
HMS_shp = pd.merge(county_shape, HMS, on = 'COUNTY')

#COUNTY POLYGONS with LOW MOISTURE DATA (using geographic center, not population)
LMS_shp = pd.merge(county_shape, LMS, on = 'COUNTY')

#COUNTY POLYGONS with ORCHARD RESIDUE DATA (using geographic center, not population)
orchres_shp = pd.merge(county_shape, orchres, on = 'COUNTY')


In [88]:
# POPULATION-WEIGHTED COUNTY CENTROIDS with BIOMASS DATA
df_pts = pd.merge(counties_popcen, df, on = 'COUNTY')

df_pts

###turn into geodataframe###
gdf_pts = GeoDataFrame(df_pts, geometry='county_centroid')

gdf_pts = gdf_pts.set_crs('epsg:4326') #{'init' :'epsg:4326'}

gdf_pts


,STATEFP,COUNTYFP,COUNTY,STNAME,POPULATION,lat,lon,county_centroid,LUMBER,PAPER,CARDBOARD,GREEN,FOOD,FOG,OTHER,food_wt,gw_wt
0,6,1,Alameda,California,1510271,37.705201,-122.100393,POINT (-122.10039 37.70520),147821,127884,33169,94768,43944,156,93735,1.464800e+05,189536.0
1,6,3,Alpine,California,1175,38.728298,-119.847603,POINT (-119.84760 38.72830),114,161,42,119,55,0,119,1.833333e+02,238.0
2,6,5,Amador,California,38091,38.394208,-120.775729,POINT (-120.77573 38.39421),3436,2971,771,2202,1021,4,2177,3.403333e+03,4404.0
3,6,7,Butte,California,220000,39.669037,-121.707635,POINT (-121.70764 39.66904),20819,20190,5237,14962,6938,22,14868,2.312667e+04,29924.0
4,6,9,Calaveras,California,45578,38.163627,-120.633718,POINT (-120.63372 38.16363),4302,3574,927,2649,1228,5,2591,4.093333e+03,5298.0
5,6,11,Colusa,California,21419,39.156148,-122.085557,POINT (-122.08556 39.15615),2134,2391,620,1772,822,2,1720,2.740000e+03,3544.0
6,6,13,Contra Costa,California,1049025,37.935588,-122.030131,POINT (-122.03013 37.93559),102511,79163,20532,58663,27202,108,58003,9.067333e+04,117326.0
7,6,15,Del Norte,California,28610,41.791232,-124.163088,POINT (-124.16309 41.79123),2561,1846,479,1368,634,3,1387,2.113333e+03,2736.0
8,6,17,El Dorado,California,181058,38.746543,-120.744330,POINT (-120.74433 38.74654),16769,15453,4008,11452,5310,18,11492,1.770000e+04,22904.0
9,6,19,Fresno,California,930450,36.738918,-119.767884,POINT (-119.76788 36.73892),92708,84538,21926,62647,29049,98,60903,9.683000e+04,125294.0


In [90]:
###export file - MSW###
print("exporting shapefile")
# out = r"/Users/sashaponomareva/Documents/compostsiting/data/feedstocks_cleaned/msw_2020.shp"
out = opj(DATA_DIR,"feedstocks_cleaned/msw_2020.shp")

gdf_pts.to_file(driver='ESRI Shapefile', filename=out)

print("p BIOMASS PRE_PROCESSING DONE RUNNING")



exporting shapefile
p BIOMASS PRE_PROCESSING DONE RUNNING


/Users/anayahall/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  


In [ ]:
print(manure_shp.columns)

###MAKE SURE IT WORKS###
manure_shp
#HMS_shp
#LMS_shp
#orchres_shp


In [ ]:
manure_shp

manure_gdf = GeoDataFrame(manure_shp, geometry='county_centroid')

manure_gdf = manure_gdf.set_crs('epsg:4326') #{'init' :'epsg:4326'}

manure_gdf.crs

manure_gdf


###export file - manure###
print("exporting shapefile")
out = r"/Users/sashaponomareva/Documents/compostsiting/data/feedstocks_cleaned/manure.shp"

manure_gdf.to_file(driver='ESRI Shapefile', filename=out)

#print("p BIOMASS PRE_PROCESSING DONE RUNNING")


###IT WORKS WHEN I REMOVE GEOMETRY AS A COLUMN###

In [ ]:
HMS_shp

HMS_gdf = GeoDataFrame(HMS_shp, geometry='county_centroid')

HMS_gdf = HMS_gdf.set_crs('epsg:4326') #{'init' :'epsg:4326'}

HMS_gdf.crs

HMS_gdf

###export file - HMS###
print("exporting shapefile")

out = r"/Users/sashaponomareva/Documents/compostsiting/data/feedstocks_cleaned/HMS.shp"

HMS_gdf.to_file(driver='ESRI Shapefile', filename=out)

In [ ]:
LMS_shp

LMS_gdf = GeoDataFrame(LMS_shp, geometry='county_centroid')

LMS_gdf = LMS_gdf.set_crs('epsg:4326') #{'init' :'epsg:4326'}

LMS_gdf.crs

LMS_gdf

###export file - HMS###
print("exporting shapefile")

out = r"/Users/sashaponomareva/Documents/compostsiting/data/feedstocks_cleaned/LMS.shp"

LMS_gdf.to_file(driver='ESRI Shapefile', filename=out)

In [ ]:
orchres_shp

orchres_gdf = GeoDataFrame(orchres_shp, geometry='county_centroid')

orchres_gdf = orchres_gdf.set_crs('epsg:4326') #{'init' :'epsg:4326'}

orchres_gdf.crs

orchres_gdf

###export file - HMS###
print("exporting shapefile")

out = r"/Users/sashaponomareva/Documents/compostsiting/data/feedstocks_cleaned/orchres.shp"

orchres_gdf.to_file(driver='ESRI Shapefile', filename=out)